## Using SQLModel to work with database tables

# <span style="color:red">clear all output before saving: db output contains passwords! </span>

this walks through process of

 - importing correct modules
 - creates a temporary sqlite db

*things to do/fix*

- use Postgresql as well as sqlite db
- get the 'get_session' working instead of instantiating a Session directly to capture tha function in one place 
    - figure out how to use yield properly
- work with an existing db URL rather than create it every time
- OR create one as needed e.g. for testing
- specify the db file to work depending on context, one for dev, prod, testing like Rails
- return True/False from initdb , test for success
- don't use globals! 

## Next steps

1. convert all to tests and delete this 
2. incorporate weath collection

In [ ]:

from ewxpwsdb.db.database import Session, init_db, engine, get_db_url
from ewxpwsdb.db.models import WeatherStation, Reading, StationType
from ewxpwsdb.db.importdata import import_station_file, read_station_table

In [ ]:
db_url = get_db_url()
db_url

In [ ]:
init_db()

In [ ]:
s = Session(engine)
objects = [
    StationType(station_type= 'DAVIS' ),
    StationType(station_type= 'SPECTRUM' ), 
    StationType(station_type= 'ONSET'),
    StationType(station_type= 'RAINWISE'),
    StationType(station_type= 'ZENTRA'),
]
s.bulk_save_objects(objects)
s.commit()

In [ ]:
station_file = '../data/test_stations.tsv'
# import, using the global 'engine' defined in database module
import_station_file(station_file)


In [ ]:
from sqlmodel import select
with Session(engine) as session:
        statement = select(WeatherStation)
        results = session.exec(statement)
        stations = results.all()

In [ ]:
len(stations)

In [ ]:
# attempt to add a duplicate
station_list = read_station_table(station_file)
dup_ws = WeatherStation.model_validate(station_list[2])
with Session(engine) as session:
    session.add(dup_ws)
    session.commit()


In [ ]:
our_table_names = ['WeatherStation', 'Reading']
from sqlalchemy import inspect
inspector = inspect(engine)

db_tables = list(inspector.get_table_names())
db_tables

### Example SQL

In [ ]:
from sqlmodel import text

sql="select * from weatherstation inner join stationtype on weatherstation.station_type = stationtype.station_type where stationtype.station_type = 'ZENTRA';"
with Session(engine) as session:
    stmt = text(sql)
    results = session.exec(stmt)
    records = results.all()

len(records)


### Clean up

remove test databases

In [ ]:
# if sqlite
import re
if re.match('sqlite', get_db_url()):
    from os import remove
    remove('ewxpws.db')

In [ ]:
# if postgresl
import re
from sqlmodel import delete, text
drop_stmt = text("""drop database ewxpws""")

if re.match('postgres', get_db_url()):
    with Session(engine) as session:
        session.exec(delete(Reading))
        session.exec(delete(StationType))
        session.exec(delete(WeatherStation))
        session.commit()

    # con = engine.connect()
    # con.execute(drop_stmt)
    # con.close()


In [ ]:
session.close()
engine.dispose()